In [1]:
from openstack import connection
from random import randint
import copy
from collections import Counter

In [2]:
def connection_to_cluster(project):
    conn = connection.Connection(auth_url="http://controller:35357/v3",
                                 project_name=project,
                                 project_domain_name="default",
                                 user_domain_name="default",
                                 username="admin",
                                 password="QIP@t4Ay")
    return conn

def get_servers(conn):
    a = [i for i in conn.compute.servers(all_tenants=True)]
    return a

def create_real_flavor_dict(maxram=1000000):
    
    fl = [[fId, conn.compute.get_flavor(flavor=fId).vcpus, conn.compute.get_flavor(flavor=fId).ram]\
               for fId in set([y.flavor['id'] for y in servers_list]) \
               if conn.compute.get_flavor(flavor=fId).ram < maxram]
    flu = [list(x) for x in set(tuple(x) for x in fl)]
    fd ={}
    for i in range(0,len(flu)):
        fd[flu[i][0]]=[flu[i][1],flu[i][2]]
        
        
    return fd

def get_hypervisors(conn):
    a = [i for i in conn.compute.hypervisors()]
    return a

In [3]:
def hyper_dict_create(c,r,hl,sl):
    'создаем  словарь гипервизоров'
    rd= {}
    for i in hl:
        h_vm_list = [s.name for s in sl if s.hypervisor_hostname == i.name]
        rd[i.name] = { 'name' : i.name,\
                         'cpu'  : c,\
                         'ram'  : r,\
                         'vms'  : h_vm_list,\
                         'vms_count' : len(h_vm_list)


            }
    return rd

def vm_dict_create(sl,fl):
    'создаем словарь виртуальных машин'
    
    rd = {}
    for i in sl:
        #print(i.name)
        res_list = fl[i.flavor['id']]
        rd[i.name] = {'name': i.name,\
                 'cpu' : res_list[0],\
                 'ram' : res_list[1],\
                 'name': i.name


        }
    return rd


def hyper_dict_resources_create(hd,vd):
    rd = copy.deepcopy(hd)
    for i in rd.keys():
        sum_ram_perc = (rd[i]['ram'] - sum([vd[j]['ram'] for j in rd[i]['vms']]))/rd[i]['ram'] 
        sum_cpu_perc = (rd[i]['cpu'] - sum([vd[j]['cpu'] for j in rd[i]['vms']]))/rd[i]['cpu']
        rd[i]['perc_cpu'] = sum_cpu_perc
        rd[i]['perc_ram'] = sum_ram_perc
    return rd

In [4]:
def count_over (h,v):
    vm_vcpu_sum = sum([v[i]['cpu'] for i in v.keys()])
    vm_ram_sum = sum([v[i]['ram'] for i in v.keys()])
    hyper_vcpu_sum = sum([h[i]['cpu'] for i in h.keys()])
    hyper_ram_sum = sum([h[i]['ram'] for i in h.keys()])
    
    return [vm_vcpu_sum, vm_ram_sum], [hyper_vcpu_sum,hyper_ram_sum],\
        [vm_vcpu_sum / hyper_vcpu_sum, vm_ram_sum/hyper_ram_sum] 

In [5]:

%%time
conn = connection_to_cluster('admin')
servers_list = get_servers(conn)
hyper_list = get_hypervisors(conn)
flavor_dict = create_real_flavor_dict()

CPU times: user 2.06 s, sys: 92.5 ms, total: 2.15 s
Wall time: 23.7 s


In [37]:
#### Описываем ВУ

cpu_per_cn = 56
#ram_per_cn = 516990
ram_per_cn = 493040
test_machine = 'site-liferay-10'
test_hypervisor = 'cn-d720.egron.tnx'

In [34]:
hDict=hyper_dict_create(cpu_per_cn,ram_per_cn,hyper_list,servers_list)
vDict=vm_dict_create(servers_list,flavor_dict)
hDictRes = hyper_dict_resources_create(hDict,vDict)
vm_res_total_list, hyper_res_total_list, over_total_list = count_over(hDictRes,vDict)

In [35]:
o = [[i,float("{0:.2f}".format(100*(1 - hDictRes[i]['perc_ram'])))] for i in hDictRes.keys() if hDictRes[i]['perc_ram'] < 0.05]
sorted(o, key = lambda x: float(x[1]),reverse=True)
len(o)

44

In [36]:
l = [hDictRes[h]['perc_ram'] for h in hDictRes.keys()]
m = [hDictRes[h]['perc_cpu'] for h in hDictRes.keys()]



print("Всего машин:", len(servers_list))
print("Необходимо машинам vCPU :" , vm_res_total_list[0], ", RAM :", vm_res_total_list[1], "MiB" )
print("На одном ВУ CPU :", cpu_per_cn, ", RAM :", ram_per_cn, "MiB")
print("Всего ВУ:", len(hyper_list))
print("Суммарно на ВУ :" , hyper_res_total_list[0], ", RAM :", hyper_res_total_list[1], "MiB" )
print("Суммарная подписка по CPU :", over_total_list[0], ", RAM :", over_total_list[1])

print("Забитость ВУ по RAM")
print("Самый забитый :", 1-min(l))
print("Наименее забитый :", 1-max(l))
print("Средняя загруженность :",  1 - sum(l)/len(l))
print('')
print("Перерасход vCPU относительно CPU ВУ ")
print("Самый забитый :", 1-min(m))
print("Наименее забитый :", 1-max(m))
print("Средняя загруженность :", 1 - sum(m)/len(m))


print("Свободно RAM :",  hyper_res_total_list[1] - vm_res_total_list[1] ," MiB")


Всего машин: 1675
Необходимо машинам vCPU : 13484 , RAM : 38395224 MB
На одном ВУ CPU : 56 , RAM : 516990 MB
Всего ВУ: 76
Суммарно на ВУ : 4256 , RAM : 39291240 MB
Суммарная подписка по CPU : 3.168233082706767 , RAM : 0.9771955275527064
Забитость ВУ по RAM
Самый забитый : 1.3706415984835296
Наименее забитый : 0.8989574266426816
Средняя загруженность : 0.9784464934168532

Перерасход vCPU относительно CPU ВУ 
Самый забитый : 4.178571428571429
Наименее забитый : 2.0357142857142856
Средняя загруженность : 3.171992481203007
Свободно RAM : 896016  MiB


In [26]:
i = conn.compute.get_hypervisor(1)
i.memory_


37246

In [31]:
ooo = [[conn.compute.get_hypervisor(i.id).name, conn.compute.get_hypervisor(i.id).memory_used, conn.compute.get_hypervisor(i.id).memory_size, conn.compute.get_hypervisor(i.id).memory_free] for i in conn.compute.hypervisors()]

In [32]:
ooo

[['cn-dcaa.egron.tnx', 479744, 516990, 37246],
 ['cn-db38.egron.tnx', 504320, 516990, 12670],
 ['cn-da22.egron.tnx', 492032, 516990, 24958],
 ['cn-e4b8.egron.tnx', 487400, 516990, 29590],
 ['cn-dca6.egron.tnx', 508416, 516990, 8574],
 ['cn-daa0.egron.tnx', 508416, 516990, 8574],
 ['cn-db26.egron.tnx', 492032, 516990, 24958],
 ['cn-dce8.egron.tnx', 659968, 516990, 0],
 ['cn-dca2.egron.tnx', 508416, 516990, 8574],
 ['cn-e4c2.egron.tnx', 487936, 516990, 29054],
 ['cn-e0ba.egron.tnx', 500224, 516990, 16766],
 ['cn-db3e.egron.tnx', 508416, 516990, 8574],
 ['cn-dc8c.egron.tnx', 483840, 516990, 33150],
 ['cn-db28.egron.tnx', 520704, 516990, 0],
 ['cn-dc8e.egron.tnx', 483840, 516990, 33150],
 ['cn-daa6.egron.tnx', 496128, 516990, 20862],
 ['cn-e13a.egron.tnx', 479744, 516990, 37246],
 ['cn-dca4.egron.tnx', 483840, 516990, 33150],
 ['cn-dc84.egron.tnx', 491936, 516989, 25053],
 ['cn-dce2.egron.tnx', 557568, 516990, 0],
 ['cn-db08.egron.tnx', 465264, 516990, 51726],
 ['cn-e4bc.egron.tnx', 475648